In [1]:
import control as ct
import cvxpy as cp
import numpy as np

### Autonomous ground vehicle

In [2]:
# Constants
M   = 1476  # [Kg]
l_f = 1.13  # [m]
l_r = 1.49  # [m]
I_e = 442.8 # [kgm^2]
I_z = 1810  # [kgm^2]
C_f = 57000 # [N/rad]
C_r = 59000 # [N/rad]
C_x = 0.35
C_y = 0.45

v_x = [5, 30]       # [m/s]
v_y = [-1.5, 1.5]   # [m/s]
r_interval   = [-0.55, 0.55] # [rad/s]

Ts = 0.01 # [s]

### Continuous Time Matrices

In [3]:
A_xi = lambda V_x, r: np.array([
    [0, r, 0],
    [0, -2*(C_f + C_r)/(M*(1/V_x)), 0],
    [0, 2*(l_r*C_r - C_f*l_f)/(I_z*(1/V_x)), 0]
])

f_v_xi = lambda V_x, r: np.array([
    -C_x*(1/V_x)**2/I_e,
    2*(C_r*l_r - C_f*l_f)*r/(M*(1/V_x)) - (1/V_x)*r,
    -2*(C_f*l_f**2 + C_r*l_r**2)*r/(I_z*(1/V_x))
]).T

V_x_interval = np.sort(1/np.array(v_x))
A_cell = []
# f_v_cell = []

for V_x_i in V_x_interval:
    for r_i in r_interval:
        A_cell.append(A_xi(V_x_i, r_i))
        # f_v_cell.append(f_v_xi(V_x_i, r_i))

E_v = np.array([
    [1/I_e, 0],
    [0, 2*C_f/M],
    [0, 2*l_f*C_f/I_z]
])

C = np.array([
    [1, 0, 0],
    [0, 0, 1]
])

G_v = np.array([0, -C_y/M, 0]).T

E_w = np.array([1, 0, 0]).T

#### Discrete Time Matrices

In [4]:
for i in range(len(A_cell)):
    A_cell[i] = Ts * A_cell[i] + np.eye(A_cell[i].shape[0])
    # f_v_cell[i] = Ts * f_v_cell[i]
f_v_xi_d = lambda V_x, r: Ts*f_v_xi(V_x, r)
E_d = Ts * E_v
G = Ts * G_v
# article does not present a conversion for E_w

#### Membership functions

In [5]:
omega_v1 = lambda V_x: (V_x_interval[1] - V_x)/(V_x_interval[1] - V_x_interval[0])
omega_v2 = lambda V_x: (V_x - V_x_interval[0])/(V_x_interval[1] - V_x_interval[0])
omega_r1 = lambda r: (r_interval[1] - r)/(r_interval[1] - r_interval[0])
omega_r2 = lambda r: (r - r_interval[1])/(r_interval[1] - r_interval[0])

h_xi = lambda V_x, r: [
    omega_v1(V_x)*omega_r1(r), 
    omega_v1(V_x)*omega_r2(r), 
    omega_v2(V_x)*omega_r1(r), 
    omega_v2(V_x)*omega_r2(r)
]

# def defuzzy(V_x, r):
#     A_h = 0
#     G_h = 0

#     h_k = h(V_x, r)

#     for i in range(len(h_k)):
#         A_h += h_k[i] * A_cell[i]
#         G_h += h_k[i] * G

#     return [A_h, G_h]

### H-$\infty$ Filter Design

### Numerical Example Dynamics

In [6]:
def vehicle_simulation(k, x, c, R, A_cell, E_d, C, G, E_w, L_h = None):
    # x  = [v_x, v_y, r]
    # xi = [1/v_x, r]

    # inputs
    u = 0
    w = 0
    
    # membemship update
    h = h_xi(1/x[0], x[2])
    f_xi = f_v_xi_d(1/x[0], x[2])
    A_h = 0
    G_h = 0

    phi_x = lambda x: x[1]**2
    
    # Model Simulation
    x_plus = 0
    for i in range(len(h)):
        x_plus += h[i]*A_cell[i]@x + E_d@u + f_xi[i] + G@phi_x(x) + E_w@w
        A_h += h[i] * A_cell[i]
        G_h += h[i] * G
    y = C@x

    # Estimator gain design
    if L_h is None:
        # calculate using online optimization
        pass
    # else use gain provided by H-infty filter
    
    # State estimation
    A_hat = A_h - L_h@C
    # retrive R_theta
    # c_w and R_w unknown -> give any bounded value to R_w -> an interval set may be enough?

    c_plus = A_hat@c + E_d@u + f_xi + G_h@phi_x(c) + L_h@y
    R_plus = np.array([A_hat@R, G_h*R_theta, E_w@R_w, 0]) # problably gonna have to fix this 0 to be a matrix

    # R reduction

    return [x_plus, c_plus, R_plus]